In [2]:
%matplotlib inline
import pandas as pd
import json
import numpy as np
from datetime import datetime, date, timedelta

# Calibration

This notebook can be used to set the baseline parameters of the model. It generates the base parameter.json file that is used for all simulations.

## 1 Set the general simulation parameters 

In [22]:
i = 0
TIME = 350
AGENTS = 100000 
CITY = ['cape_town', 'johannesburg'][i]
REGION = ['Western Cape', 'Gauteng'][i]
POPULATIONS2011 = [3740000, 4435000]
POPULATIONS2019 = [4524000, 5635127]

## 2 Set the start and end dates for the validation period

In [23]:
city_dates = ['2020-04-17', '2020-04-29'] 
START_DATE = datetime.strptime(city_dates[i], '%Y-%m-%d')
END_DATE = datetime.strptime('2020-08-10', '%Y-%m-%d') 

## 3 Import data

### 3.1 Oxford Stringency Index

In [24]:
stringency_index = pd.read_csv('general_data/OxCGRT_latest.csv')[pd.read_csv('general_data/OxCGRT_latest.csv')['CountryCode'] == 'ZAF']
stringency_index.index = [datetime.strptime(str(x), '%Y%m%d') for x in stringency_index['Date']]
stringency_index = stringency_index['StringencyIndex']

In [25]:
lockdown_severeness = stringency_index.loc[START_DATE:END_DATE]

### 3.2 Google mobility data

In [26]:
mobility_data = pd.read_csv('general_data/Global_Mobility_Report_ZA.csv')
mobility_data = mobility_data[mobility_data['sub_region_1'] == REGION]
mobility_data.index = [datetime.strptime(x, '%Y-%m-%d') for x in mobility_data['date']]
mobility_data = mobility_data[mobility_data.columns[9:]].astype(float)

### Excess fatalities

In [27]:
ef = pd.read_csv('general_data/excess_death_curves.csv')

In [28]:
ef_jhn = []
ef_ct = []

for name, l in zip(['excess_d_ct', 'excess_d_jhn'], [ef_ct, ef_jhn]):
    # remove nan value
    for x in ef[name].iloc[:117]:
        if str(x) != 'nan':
            l.append(float(x))
        else:
            l.append(0.0)

In [29]:
excess_fatalities = [ef_ct, ef_jhn]

## 3 Set policy parameters for the duration of the simulation.
Policy parameters are input as a list that is as long as the simulation. This way they can change over the course of the simulation, in line with observed policy. 

The travel multiplier is set using the Google mobility data.

In [30]:
DATE = '2020-03-27'

In [31]:
travel_multiplier = list(1 + mobility_data.mean(axis=1).loc[DATE:DATE] / 100)[0]
travel_multiplier

0.4066666666666666

## 4 Set initial infections and age groups

Next, we assume that 3% of infections were detected at the start of the simulation and translate this to the initial number of cases at the start of the simulation. 

In [32]:
perc_infections_detects = 3
initial_agents = max(round((310 / (POPULATIONS2019[i] / AGENTS) * 100 / perc_infections_detects)), 20) # 310 cases / (population / agent) * 1 / 14% detected cases
initial_agents

228

The age groups are per decile. 

In [33]:
age_groups = ['age_0_10', 'age_10_20', 'age_20_30', 'age_30_40', 'age_40_50', 
              'age_50_60', 'age_60_70', 'age_70_80', 'age_80_plus']

Health system capacity city 

In [34]:
beds_joburg = 8750 / POPULATIONS2019[1]
beds_cape_town = 0.0009179
health_system_capacities = [beds_cape_town, beds_joburg]
health_system_capacities

[0.0009179, 0.0015527600353993796]

## 5 Create the parameters 

In [35]:
parameters = {
    # Parameters related to model implementation
    "time": TIME, 
    "number_of_agents": AGENTS,
    
    # COVID-19 parameters (9)
    "exposed_days": 4, # (not changed) average number of days before being able to infect others (sources: NICD + CDC)
    "asymptom_days": 7, # (used to be 10) average number of days agents are infected but do not have symptoms 
    "symptom_days": 7,# (used to be 10) average number of days agents with mild symptoms are infectious (NICD = 7, Balabdaoui and Mohr = 8, Huang et al=7)
    "critical_days": 11, # (used to be 8) average number of days agents are in critical condition (Balabdaoui and Mohr = 8, NICD=8-19 (13.5), CDC=10-14 (12))
    "probability_symptomatic": (1 - 0.6165), # (not changed) determines whether an agent will become asymptomatic or asymptomatic spreader
    "no_hospital_multiplier": 1.79, # the increase in probability if a critical agent cannot go to the hospital SOURCE: Zhou et al. 2020
    "probability_transmission": 0.01610378740708691, # the probability that the virus is transmitted when two agents interact
    "probability_critical": {key:value for key, value in zip(age_groups, [0.001, 0.003, 0.012, 0.032, 0.049, 0.102, 0.166, 0.244, 0.273])}, # probability that an agent enters a critical stage of the disease SOURCE: Verity et al.
    "probability_to_die": {key:value for key, value in zip(age_groups, [0.02090209, 0.032569361, 0.034233668, 0.052638239, 0.097470817, 0.155112718, 0.248512233, 0.306164902, 0.371187541])}, #used to be [0.005, 0.021, 0.053, 0.126, 0.221, 0.303, 0.565, 0.653, 0.765])}, probability to die per age group in critical stage SOURCE: Verity et al.
    
    # learning parameters
    'private_shock_stdev': 0.05,  # the standard deviation for a truncated normal distribution shock that is part of the private signal for the deGroot learning used by the agents. 
    'weight_private_signal': 0.15,  # the weight of the private signal vis à vis the social signal, used in the deGroot learning process.
    
    # Cape Town specific parameters (2)
    "health_system_capacity": health_system_capacities[i],
    "stringency_index": list(lockdown_severeness),
    # Reducing travel e.g. by reducing it for work, school or all
    "visiting_recurring_contacts_multiplier": travel_multiplier,#[travel_multiplier for x in range(0, TIME)], # based on travel data
    
    # initial infections
    "total_initial_infections": initial_agents, # total agents infected in CT
    
    # optional parameters for second wave
    'time_4_new_infections': -1, # -1 is never
    'new_infections_scenario': 'None', # determines where the initial infections will be if either initial (infections will pop up in the same place as initially), or random (infections pop up in random districts). Alternatively, this parameter is None and then no second re-seeding will occur. 
    
    # additional parameter used to switch of informal districts
    "informality_dummy": 1.0, # setting this parameter at 0 will mean the lockdown is equally effective anywhere, alternative = 1
    
    # Technical parameters
    'init_infected_agent': 0, # to calculate R0
    "data_output": 'csv-light', # 'csv', 'csv-light' or 'network', or 'False'
    
    # parameters used for comparing to data
    'empirical_population': POPULATIONS2019[i],  # specifies the population for the city that is modelled. 
    'empirical_fatalities': excess_fatalities[i],  # 
    
    # Depreciated paramters (can be used later)
    "probability_susceptible": 0.000, # probability that the agent will again be susceptible after having recovered
}

## Next, we store these parameters in a .json file.

In [36]:
with open('{}/parameters.json'.format(CITY), 'w') as outfile:
    json.dump(parameters, outfile)

In [22]:
# with open('config_{}.json'.format(CITY), 'w') as outfile:
#     json.dump(parameters, outfile)